In [2]:
import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F

In [3]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [4]:
transform = transforms.Compose(
        [transforms.Resize((256,256)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 500

trainset_big = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

trainset = torch.utils.data.Subset(trainset_big,list(range(10000)))

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

val_set = torch.utils.data.Subset(trainset_big,list(range(10000, 20000)))
val_loader = torch.utils.data.DataLoader(val_set, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [5]:
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs):
    best_model = model
    best_loss = 100
    for epoch in range(num_epochs):
        tr_correct = 0
        tr_total = 0
        val_correct = 0
        val_total = 0
        for batch_nr, (data, labels) in enumerate(train_loader):
            
            print("Epoch: ",epoch,"Batch: ",batch_nr)
            # calculate prediction according to our model
            
            prediction = model.forward(data)
            # Calculate the loss of the prediction by comparing to the expected output
            loss = criterion(prediction, labels)
            
            # Backpropagate the loss through the network to find the gradients of all parameters
            loss.backward()
            
            # Update the parameters along their gradients
            optimizer.step()
            
            # Clear stored gradient values
            optimizer.zero_grad()

            # calculate accuracy
            for i in range(len(data)):    
                guess = torch.argmax(prediction[i])
                if(guess.item() == labels[i]):
                    tr_correct+=1
                tr_total +=1



        for batch_nr, (data, labels) in enumerate(val_loader):
            
            prediction = model.forward(data)
            
            # Calculate the loss of the prediction by comparing to the expected output
            loss = criterion(prediction, labels)

            if(loss < best_loss):
                best_loss = loss
                best_model = model

            # calculate accuracy
            for i in range(len(data)):    
                guess = torch.argmax(prediction[i])
                if(guess.item() == labels[i]):
                    val_correct+=1
                val_total +=1

    # primt accuracy
    tr_accuracy = tr_correct/tr_total
    val_accuracy = val_correct/val_total
    print(f'Training accuracy:   {str(100*tr_accuracy)[:4]}%.')
    print(f'Validation accuracy: {str(100*val_accuracy)[:4]}%.')

    return best_model

In [6]:
def test_model(model, test_loader):
    val_correct = 0
    val_total = 0
    for batch_nr, (data, labels) in enumerate(test_loader):
        # print("ohno")
        prediction = model.forward(data)
        
        # calculate accuracy
        for i in range(len(data)):    
            guess = torch.argmax(prediction[i])
            if(guess.item() == labels[i]):
                val_correct+=1
            val_total +=1

    # primt accuracy
    val_accuracy = val_correct/val_total
    print(f'Test accuracy: {str(100*val_accuracy)[:4]}%.')

In [52]:
alexNet_fineTuning = torchvision.models.alexnet(pretrained=True)
alexNet_fineTuning.classifier[6] = nn.Linear(4096,10)

# alexNet_fineTuning.eval

In [53]:
# Define for the optimization algorithm which parameters we want to update during training

# Define our loss function
criterion = torch.nn.CrossEntropyLoss()

# Define our optimizer
optimizer = torch.optim.Adam(alexNet_fineTuning.parameters(), lr=0.001)

In [54]:
# Run the training step
train_model(alexNet_fineTuning,criterion,optimizer,trainloader,val_loader,num_epochs=10)

Epoch:  0 Batch:  0
Epoch:  0 Batch:  1
Epoch:  0 Batch:  2
Epoch:  0 Batch:  3
Epoch:  0 Batch:  4
Epoch:  0 Batch:  5
Epoch:  0 Batch:  6
Epoch:  0 Batch:  7
Epoch:  0 Batch:  8
Epoch:  0 Batch:  9
Epoch:  0 Batch:  10
Epoch:  0 Batch:  11
Epoch:  0 Batch:  12
Epoch:  0 Batch:  13
Epoch:  0 Batch:  14
Epoch:  0 Batch:  15
Epoch:  0 Batch:  16
Epoch:  0 Batch:  17
Epoch:  0 Batch:  18
Epoch:  0 Batch:  19
Epoch:  1 Batch:  0
Epoch:  1 Batch:  1
Epoch:  1 Batch:  2
Epoch:  1 Batch:  3
Epoch:  1 Batch:  4
Epoch:  1 Batch:  5
Epoch:  1 Batch:  6
Epoch:  1 Batch:  7
Epoch:  1 Batch:  8
Epoch:  1 Batch:  9
Epoch:  1 Batch:  10
Epoch:  1 Batch:  11
Epoch:  1 Batch:  12
Epoch:  1 Batch:  13
Epoch:  1 Batch:  14
Epoch:  1 Batch:  15
Epoch:  1 Batch:  16
Epoch:  1 Batch:  17
Epoch:  1 Batch:  18
Epoch:  1 Batch:  19
Epoch:  2 Batch:  0
Epoch:  2 Batch:  1
Epoch:  2 Batch:  2
Epoch:  2 Batch:  3
Epoch:  2 Batch:  4
Epoch:  2 Batch:  5
Epoch:  2 Batch:  6
Epoch:  2 Batch:  7
Epoch:  2 Batch:  8


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [12]:
# Define for the optimization algorithm which parameters we want to update during training
alexNet_featureExtraction = torchvision.models.alexnet(pretrained=True)

for param in alexNet_featureExtraction.parameters():
    param.requires_grad = False

alexNet_featureExtraction.classifier[6] = nn.Linear(4096,10)
# find the paramaters we want to update during training
params_to_update = []
for param in alexNet_featureExtraction.parameters():
    if param.requires_grad == True:
        params_to_update.append(param)

for name, param in alexNet_featureExtraction.named_parameters():
    if param.requires_grad:
        print (name, param.data)

# Define our loss function
criterion_fe = torch.nn.CrossEntropyLoss()

# Define our optimizer
optimizer_fe = torch.optim.Adam(params_to_update, lr=0.001)

classifier.6.weight tensor([[ 0.0061, -0.0003, -0.0059,  ...,  0.0097,  0.0109,  0.0074],
        [ 0.0108,  0.0139, -0.0114,  ..., -0.0045, -0.0047,  0.0107],
        [-0.0038,  0.0126, -0.0131,  ...,  0.0013, -0.0023, -0.0046],
        ...,
        [-0.0079, -0.0107,  0.0092,  ..., -0.0069, -0.0136, -0.0094],
        [ 0.0065,  0.0043, -0.0061,  ..., -0.0052,  0.0021, -0.0071],
        [ 0.0156, -0.0142, -0.0107,  ..., -0.0050,  0.0037, -0.0087]])
classifier.6.bias tensor([-0.0078, -0.0155, -0.0026, -0.0136, -0.0021,  0.0057,  0.0017, -0.0139,
        -0.0072,  0.0117])


In [13]:
# Run the training step
train_model(alexNet_featureExtraction,criterion_fe,optimizer_fe,trainloader,val_loader,num_epochs=10)

Epoch:  0 Batch:  0
Epoch:  0 Batch:  1
Epoch:  0 Batch:  2
Epoch:  0 Batch:  3
Epoch:  0 Batch:  4
Epoch:  0 Batch:  5
Epoch:  0 Batch:  6
Epoch:  0 Batch:  7
Epoch:  0 Batch:  8
Epoch:  0 Batch:  9
Epoch:  0 Batch:  10
Epoch:  0 Batch:  11
Epoch:  0 Batch:  12
Epoch:  0 Batch:  13
Epoch:  0 Batch:  14
Epoch:  0 Batch:  15
Epoch:  0 Batch:  16
Epoch:  0 Batch:  17
Epoch:  0 Batch:  18
Epoch:  0 Batch:  19
Epoch:  1 Batch:  0
Epoch:  1 Batch:  1
Epoch:  1 Batch:  2
Epoch:  1 Batch:  3
Epoch:  1 Batch:  4
Epoch:  1 Batch:  5
Epoch:  1 Batch:  6
Epoch:  1 Batch:  7
Epoch:  1 Batch:  8
Epoch:  1 Batch:  9
Epoch:  1 Batch:  10
Epoch:  1 Batch:  11
Epoch:  1 Batch:  12
Epoch:  1 Batch:  13
Epoch:  1 Batch:  14
Epoch:  1 Batch:  15
Epoch:  1 Batch:  16
Epoch:  1 Batch:  17
Epoch:  1 Batch:  18
Epoch:  1 Batch:  19
Epoch:  2 Batch:  0
Epoch:  2 Batch:  1
Epoch:  2 Batch:  2
Epoch:  2 Batch:  3
Epoch:  2 Batch:  4
Epoch:  2 Batch:  5
Epoch:  2 Batch:  6
Epoch:  2 Batch:  7
Epoch:  2 Batch:  8


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [55]:
print("Fine Tuning:")
test_model(alexNet_fineTuning, testloader)

print("\nFeature Extraction:")
test_model(alexNet_featureExtraction, testloader)

Fine Tuning:
Test accuracy: 35.5%.

Feature Extraction:
Test accuracy: 71.5%.


In [11]:
transform = transforms.Compose(
        [transforms.ToTensor(),
        transforms.Normalize((0.5), (0.5))])

batch_size = 500

trainset_MNIST_big = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)

trainset_MNIST = torch.utils.data.Subset(trainset_MNIST_big,list(range(10000)))

trainloader_MNIST = torch.utils.data.DataLoader(trainset_MNIST, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

val_set_MNIST = torch.utils.data.Subset(trainset_MNIST_big,list(range(10000, 20000)))
val_loader_MNIST = torch.utils.data.DataLoader(val_set_MNIST, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
                                          
testset_MNIST = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)
testloader_MNIST = torch.utils.data.DataLoader(testset_MNIST, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

In [12]:
# Chose ResNet-18 as CNN model
resnet = torchvision.models.resnet18(pretrained= False)
resnet.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
# optimizer = torch.optim.Adam(params_to_update, lr=0.001)
optimizer = torch.optim.Adam(resnet.parameters(), lr=0.001)

criterion = nn.CrossEntropyLoss()

In [13]:
# Trained the model on MNIST data
trained_resnet = train_model(resnet, criterion, optimizer, trainloader_MNIST, val_loader_MNIST, num_epochs=10)
test_model(trained_resnet, testloader_MNIST)

Epoch:  0 Batch:  0
Epoch:  0 Batch:  1
Epoch:  0 Batch:  2
Epoch:  0 Batch:  3
Epoch:  0 Batch:  4
Epoch:  0 Batch:  5
Epoch:  0 Batch:  6
Epoch:  0 Batch:  7
Epoch:  0 Batch:  8
Epoch:  0 Batch:  9
Epoch:  0 Batch:  10
Epoch:  0 Batch:  11
Epoch:  0 Batch:  12
Epoch:  0 Batch:  13
Epoch:  0 Batch:  14
Epoch:  0 Batch:  15
Epoch:  0 Batch:  16
Epoch:  0 Batch:  17
Epoch:  0 Batch:  18
Epoch:  0 Batch:  19
Epoch:  1 Batch:  0
Epoch:  1 Batch:  1
Epoch:  1 Batch:  2
Epoch:  1 Batch:  3
Epoch:  1 Batch:  4
Epoch:  1 Batch:  5
Epoch:  1 Batch:  6
Epoch:  1 Batch:  7
Epoch:  1 Batch:  8
Epoch:  1 Batch:  9
Epoch:  1 Batch:  10
Epoch:  1 Batch:  11
Epoch:  1 Batch:  12
Epoch:  1 Batch:  13
Epoch:  1 Batch:  14
Epoch:  1 Batch:  15
Epoch:  1 Batch:  16
Epoch:  1 Batch:  17
Epoch:  1 Batch:  18
Epoch:  1 Batch:  19
Epoch:  2 Batch:  0
Epoch:  2 Batch:  1
Epoch:  2 Batch:  2
Epoch:  2 Batch:  3
Epoch:  2 Batch:  4
Epoch:  2 Batch:  5
Epoch:  2 Batch:  6
Epoch:  2 Batch:  7
Epoch:  2 Batch:  8


In [15]:
transform = transforms.Compose(
        [transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 500

all_svhm_train_data = torchvision.datasets.SVHN(root='./data', split= 'train'
                                        download=True, transform=transform)
all_svhm_test_data = torchvision.datasets.SVHN(root='./data', split= 'test'
                                        download=True, transform=transform)
svhm_test_set = torch.utils.data.Subset(all_svhm_train_data,list(range(10000)))
svhm_val_set  = torch.utils.data.Subset(all_svhm_train_data,list(range(10000, 20000)))
svhm_test_set  = torch.utils.data.Subset(all_svhm_train_data,list(range(10000)))




# svhn_train_loader = torch.utils.data.DataLoader(all_svhm_data,
#                                           batch_size=batch_size,
#                                           shuffle=True,
#                                           num_workers=2)


# trainset_MNIST_big = torchvision.datasets.ImageNet(root='./data', train=True,
#                                         download=True, transform=transform)

# svhm_test_set = torch.utils.data.Subset(all_svhm_data,list(range(10000)))

# trainloader_MNIST = torch.utils.data.DataLoader(trainset_MNIST, batch_size=batch_size,
#                                           shuffle=True, num_workers=2)

# val_set_MNIST = torch.utils.data.Subset(trainset_MNIST_big,list(range(10000, 20000)))
# val_loader_MNIST = torch.utils.data.DataLoader(val_set_MNIST, batch_size=batch_size,
#                                           shuffle=True, num_workers=2)
                                          
# testset_MNIST = torchvision.datasets.MNIST(root='./data', train=False,
#                                        download=True, transform=transform)
# testloader_MNIST = torch.utils.data.DataLoader(testset_MNIST, batch_size=batch_size,
#                                          shuffle=False, num_workers=2)

TypeError: __init__() got an unexpected keyword argument 'train'

In [ ]:
# Chose ResNet-18 as CNN model
resnet_svhn = torchvision.models.resnet18(pretrained= False)
resnet.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
optimizer = torch.optim.Adam(resnet.parameters(), lr=0.001)

criterion = nn.CrossEntropyLoss()